In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Tensorflow related
import tensorflow as tf
from tensorflow import keras

# If `%matplotlib widget` does not work then use `%matplotlib notebook`
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors
import matplotlib.widgets as widgets

import numpy as np

import glob
from PIL import Image

# Loading saved training images

In [ ]:
NoImages=np.load("traindata_noimages.npy")
YesImages=np.load("traindata_yesimages.npy")
print("Shapes: {} and {}".format(NoImages.shape,YesImages.shape))

## Optional: creating new training images

- Change the directory here to the one containing the camera image dumps

In [ ]:
imageFilenames = glob.glob("framedumps/*.tga")
curImageIndex = 0

def loadImage(filename):
    npim = np.array(Image.open(filename))
    npim = np.delete(npim, 3, axis=2) # delete A from RGBA
    return npim

- Make sure the existing images are loaded in the `NoImages`,`YesImages` arrays
- Empty the `newNoImages` and `newYesImages` lists

In [ ]:
newNoImages = []
newYesImages = []

- Evaluate the cell below to get a matplotlib interactive figure and **repeat** the following:
- Click somewhere in the image to crop out a part and click one of the buttons to save it
- Click next to open a new image

In [ ]:
def onclick(event):
    # event.x,y is click coordinates but not in image space
    # event.xdata,ydata is coordinates in image
    xmin = int(event.xdata) - 20
    xmax = int(event.xdata) + 20
    ymin = int(event.ydata) - 20
    ymax = int(event.ydata) + 20
    global data
    global fig1cropped
    global croppedimg
    croppedimg = data[ymin:ymax,xmin:xmax]
    fig1cropped.imshow(croppedimg)
    
def onNextBtn(event):
    global data
    global fig1mainpic
    global curImageIndex
    global imageFilenames
    data = loadImage(imageFilenames[curImageIndex])
    fig1mainpic.imshow(data)
    curImageIndex += 1
    if curImageIndex >= len(imageFilenames):
        curImageIndex = 0

def onYesBtn(event):
    global croppedimg
    global newYesImages
    newYesImages.append(croppedimg)

def onNoBtn(event):
    global croppedimg
    global newNoImages
    newNoImages.append(croppedimg)

try:
    if cid1 is not None:
        fig1.canvas.mpl_disconnect(cid1)
    if fig1 is not None:
        plt.close(fig1)
except:
    fig1 = None
    cid1 = None
fig1 = plt.figure()
cid1 = fig1.canvas.mpl_connect('button_press_event', onclick)
fig1.canvas.layout.width ='900px'
fig1.canvas.layout.height='800px'
fig1mainpic = fig1.add_axes([0,   0.4, 1,   0.6]) # dist from left, dist from bottom, width, height
fig1cropped = fig1.add_axes([0.4, 0.1, 0.2, 0.3]) # dist from left, dist from bottom, width, height
fig1mainpic.axis('off')
fig1cropped.axis('off')
axYes  = fig1.add_axes([0.25-0.09, 0.05, 0.18, 0.05])
axNo   = fig1.add_axes([0.50-0.09, 0.05, 0.18, 0.05])
axNext = fig1.add_axes([0.75-0.09, 0.05, 0.18, 0.05])
btnYes = widgets.Button(axYes,label='Add to YES instances')
btnNo = widgets.Button(axNo,label='Add to NO instances')
btnNext = widgets.Button(axNext,label='Next image')
btnYes.on_clicked(onYesBtn)
btnNo.on_clicked(onNoBtn)
btnNext.on_clicked(onNextBtn)
onNextBtn(None) #Load the first image

- Add the new images to the list 

In [ ]:
NoImages = np.concatenate((NoImages,np.array(newNoImages,dtype=np.uint8)))

In [ ]:
YesImages = np.concatenate((YesImages,np.array(newYesImages,dtype=np.uint8)))

- Save the new array

In [ ]:
np.save("traindata_noimages.npy",NoImages)
np.save("traindata_yesimages.npy",YesImages)
print("Shapes: {} and {}".format(NoImages.shape,YesImages.shape))

# Neural network

## Choose a model

### Option 1: Load saved neural network

In [ ]:
# Load model definition and trained weights
model = keras.models.load_model("PixelNetModel_leaky.h5", custom_objects={'leaky_relu':tf.nn.leaky_relu})
model.summary()

In [ ]:
# Alternatively, define a model below and load only trained weights (into another model)
model.load_weights("PixelNetModel_leaky_weights.h5")

### Option 2: Define a new neural network

In [ ]:
# Use convolutions with a 1x1 kernel to make sure every operation happens per-pixel
model = keras.Sequential([
    #keras.layers.Conv2D(4, kernel_size=(1,1), strides=(1, 1), padding='valid', activation=lambda x:tf.nn.leaky_relu(x, alpha=0.05), input_shape=(40,40,3)),
    keras.layers.Conv2D(4, kernel_size=(1,1), strides=(1, 1), padding='valid', activation=tf.nn.leaky_relu, input_shape=(40,40,3)),
    keras.layers.Conv2D(1, kernel_size=(1,1), strides=(1, 1), padding='valid', activation=tf.nn.sigmoid),
    keras.layers.Flatten(),
    keras.layers.Lambda( lambda x : keras.backend.mean(x,axis=1,keepdims=True) ),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer=keras.optimizers.Adam(lr=0.01),
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

## Train (or continue training) the neural network

- First create and shuffle training data

In [ ]:
labels0=np.full(len(NoImages),  0, dtype=np.uint8)
labels1=np.full(len(YesImages), 1, dtype=np.uint8)
# Make sure to rescale input to [0,1] range
train_data= (1.0 / 255.0) * np.concatenate((NoImages, YesImages))
train_labels =np.concatenate((labels0,labels1))

# Shuffle
p = np.random.permutation(len(train_data))
train_data = train_data[p]
train_labels = train_labels[p]

print("Train data shape: {}".format(train_data.shape))

- Run training (can be called on a trained model to improve it further)

In [ ]:
model.fit(train_data, train_labels, epochs=1000)

- Optionally save the model

In [ ]:
model.save("PixelNetModel_1.h5")
model.save_weights("PixelNetModel_1_weights.h5")

## Visualize performance: evaluate the neural network on the image set

In [ ]:
scaledNoImages = (1.0 / 255.0) * NoImages
scaledYesImages = (1.0 / 255.0) * YesImages
# Get the answers of all images
noAnswers=model.predict(scaledNoImages).flatten().tolist()
yesAnswers=model.predict(scaledYesImages).flatten().tolist()
# Get only the first part of the model that runs per-pixel
modelpart=keras.Sequential([model.layers[0], model.layers[1]])
# Run it on the images
filteredNoImages=modelpart.predict(scaledNoImages).squeeze() # Squeeze removes the 1-dimensional axis at the end
filteredYesImages=modelpart.predict(scaledYesImages).squeeze() # Squeeze removes the 1-dimensional axis at the end

In [ ]:
# It might take a while for the plots to show if you plot many of them
numToShow = 10

def getColor(x):
    x = 0.5 + 4*(x - 0.5)**3 # Amplify the things away from 0 and 1
    return (1-x, x, 0)

try:
    plt.close(fig2)
except:
    fig2 = None

mplnorm = colors.Normalize(vmin=0, vmax=1)
fig2 = plt.figure(figsize=(6,0.8*numToShow))
fig2.canvas.layout.width = "600px"
fig2.canvas.layout.height = "{}px".format(numToShow * 80)
subplots2 = fig2.subplots(nrows=numToShow,ncols=6)
for i in range(numToShow):
    for j in range(6):
        subplots2[i,j].axis('off')
    if i < len(NoImages):
        subplots2[i,0].add_patch(patches.Rectangle((0,0),1,1,facecolor=getColor(noAnswers[i])))
        subplots2[i,0].text(0.3,0.45,"{:4.2f}".format(noAnswers[i]))
        subplots2[i,1].imshow(filteredNoImages[i], cmap=plt.cm.plasma, norm=mplnorm)
        subplots2[i,2].imshow(NoImages[i])
    if i < len(YesImages):
        subplots2[i,3].imshow(YesImages[i])
        subplots2[i,4].imshow(filteredYesImages[i], cmap=plt.cm.plasma, norm=mplnorm)
        subplots2[i,5].add_patch(patches.Rectangle((0,0),1,1,facecolor=getColor(yesAnswers[i])))
        subplots2[i,5].text(0.3,0.45,"{:4.2f}".format(yesAnswers[i]))

# Coding the Raspberry Pi tracker

- Generating GLSL shader code

The output of the next cell should be put in the file `src/tracker/balltrackshaders/colorfilterXXX.frag`

In [ ]:
w0=model.layers[0].weights[0].numpy().squeeze()
b0=model.layers[0].weights[1].numpy().squeeze()
w1=model.layers[1].weights[0].numpy().squeeze()
b1=model.layers[1].weights[1].numpy().squeeze()

# Check if we should invert the output:
# We want NO < 0 and YES > 0
if model.layers[-1].weights[0].numpy().squeeze() < 0:
    w1 = -w1;
    b1 = -b1;

def getGLSLmatrix(x,b):
    out = "mat4(\n"
    for i in range(3):
        for j in range(4):
            out += "{:9.4f},".format(x[i,j])
        out += "  // column {}\n".format(i+1)
    for j in range(3):
        out += "{:9.4f},".format(b[j])
    out += "{:9.4f}); // last column (biases)".format(b[3])
    return out

def getGLSLvec(x):
    out = "vec4("
    for i in range(3):
        out += "{:7.4f},".format(x[i])
    out += "{:7.4f});".format(x[3])
    return out

print("mat4 weights0 = {}".format(getGLSLmatrix(w0,b0)))
print("vec4 weights1 = {}".format(getGLSLvec(w1)))
print("float b1 = {:7.4f};".format(b1))

- Choose a threshold which should be used by the tracker, i.e. at what final output of the neural network should it be considered a YES. This threshold is then computed back to a number of pixels.
- The output of the next cell should be put in `src/tracker/analysis.cpp` in the function `analysis_process_ball_buffer`.

In [ ]:
threshold = 0.7

finalweight=model.layers[4].weights[0].numpy()[0,0]
finalbias=model.layers[4].weights[1].numpy()[0]
numpixels = (1600/finalweight) * (np.log(threshold/(1 - threshold)) - finalbias)
if finalweight < 0:
    numpixels = 1600 - numpixels
scalednumpixels = int(numpixels * (255/64))
print("int threshold2 = {};".format(scalednumpixels))